# 국토교통부_오피스텔 매매 신고 조회 서비스


- 데이터 출처: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058452   
- 지역코드 출처: 주민등록주소코드 변경내역(2018. 1. 22. 현재)    https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=61552 내 첨부파일 jscode20180122.zip   
주의: 일일 트래픽 1000회 제한

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

## 함수 설정

In [10]:
def collect_offi_trade(ym, lawd_cd) :
    
    # 제공받은 API 
    API_KEY = 'bHg%2FpafJkgCqoGEFC3cI5IvynqkFuwK53BYhBu3bqMz7I%2BX%2BEYH8R9jV%2FCMc06dm11DO%2FksvWzs5sIUeyCjiTQ%3D%3D'
    
    # URL
    url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiTrade'
 
    # 요청메시지
    url = url + '?serviceKey=' + API_KEY + '&LAWD_CD=' + lawd_cd + '&DEAL_YMD=' + ym 
 
    resultXML = urlopen(url)
    result = resultXML.read()
    xmlsoup = BeautifulSoup(result, 'lxml-xml')
 
    te = xmlsoup.findAll('item')
 
    # 데이터프레임 결과물
    offi_trade = pd.DataFrame()
 
    for t in te:
        deal_amt = t.find('거래금액').text

        try:
            build_y = t.find('건축년도').text
        except:
            build_y = "" 

        deal_y = t.find('년').text
        deal_m = t.find('월').text
        deal_day = t.find('일').text
        dong = t.find('법정동').text
        apt_nm = t.find('단지').text
        area_exc = t.find('전용면적').text
        
        try:
            region_num = t.find("지번").text
        except:
            region_num = ""

        region_code = t.find('지역코드').text
        floor = t.find('층').text
 
        temp = pd.DataFrame(([[deal_amt, build_y, deal_y, deal_m, deal_day, dong, apt_nm, area_exc, region_num, region_code, floor]]),
                           columns = ['deal_amt', 'build_y', 'deal_y', 'deal_m', 'deal_day', 'dong', 'apt_nm', 'area_exc', 'region_num', 'region_code', 'floor'])
        offi_trade = pd.concat([offi_trade, temp])
 
    offi_trade = offi_trade.reset_index(drop=True)
 
    return offi_trade

## 수집

#### 자료 수집 현황
2020년 data => 7/19 완료    

### **** 2019년부터 진행 예정 ****

In [14]:
if __name__ == "__main__":

    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"]=="서울특별시") | (code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    offi_trade = pd.DataFrame()

    # 기간설정(연월: 'yyyymm')
    ym = list(['201901', '201902', '201903', '201904', '201905', '201906', 
               '201907', '201908', '201909', '201910', '201911','201912'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_offi_trade(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            offi_trade = pd.concat([offi_trade, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

11110, 14개 완료
11140, 20개 완료
11170, 19개 완료
11200, 19개 완료
11215, 11개 완료
11230, 29개 완료
11260, 4개 완료
11290, 84개 완료
11305, 25개 완료
11320, 13개 완료
11350, 7개 완료
11380, 20개 완료
11410, 37개 완료
11440, 42개 완료
11470, 19개 완료
11500, 62개 완료
11530, 25개 완료
11545, 11개 완료
11560, 54개 완료
11590, 1개 완료
11620, 28개 완료
11650, 44개 완료
11680, 57개 완료
11710, 42개 완료
11740, 30개 완료
41110, 0개 완료
41111, 3개 완료
41113, 14개 완료
41115, 17개 완료
41117, 36개 완료
41130, 0개 완료
41131, 15개 완료
41133, 15개 완료
41135, 57개 완료
41150, 40개 완료
41170, 0개 완료
41171, 20개 완료
41173, 26개 완료
41190, 49개 완료
41210, 13개 완료
41220, 1개 완료
41250, 0개 완료
41270, 0개 완료
41271, 5개 완료
41273, 19개 완료
41280, 0개 완료
41281, 3개 완료
41285, 44개 완료
41287, 4개 완료
41290, 0개 완료
41310, 3개 완료
41360, 1개 완료
41370, 6개 완료
41390, 6개 완료
41410, 44개 완료
41430, 2개 완료
41450, 9개 완료
41460, 0개 완료
41461, 8개 완료
41463, 38개 완료
41465, 5개 완료
41480, 3개 완료
41500, 1개 완료
41550, 9개 완료
41570, 4개 완료
41590, 29개 완료
41610, 1개 완료
41630, 0개 완료
41650, 0개 완료
41670, 3개 완료
41800, 0개 완료
41820, 0개 완료
41830, 0개 완료
******** 2019

41110, 0개 완료
41111, 0개 완료
41113, 0개 완료
41115, 0개 완료
41117, 0개 완료
41130, 0개 완료
41131, 0개 완료
41133, 0개 완료
41135, 0개 완료
41150, 0개 완료
41170, 0개 완료
41171, 0개 완료
41173, 0개 완료
41190, 0개 완료
41210, 0개 완료
41220, 0개 완료
41250, 0개 완료
41270, 0개 완료
41271, 0개 완료
41273, 0개 완료
41280, 0개 완료
41281, 0개 완료
41285, 0개 완료
41287, 0개 완료
41290, 0개 완료
41310, 0개 완료
41360, 0개 완료
41370, 0개 완료
41390, 0개 완료
41410, 0개 완료
41430, 0개 완료
41450, 0개 완료
41460, 0개 완료
41461, 0개 완료
41463, 0개 완료
41465, 0개 완료
41480, 0개 완료
41500, 0개 완료
41550, 0개 완료
41570, 0개 완료
41590, 0개 완료
41610, 0개 완료
41630, 0개 완료
41650, 0개 완료
41670, 0개 완료
41800, 0개 완료
41820, 0개 완료
41830, 0개 완료
******** 201909 완료 ********
11110, 0개 완료
11140, 0개 완료
11170, 0개 완료
11200, 0개 완료
11215, 0개 완료
11230, 0개 완료
11260, 0개 완료
11290, 0개 완료
11305, 0개 완료
11320, 0개 완료
11350, 0개 완료
11380, 0개 완료
11410, 0개 완료
11440, 0개 완료
11470, 0개 완료
11500, 0개 완료
11530, 0개 완료
11545, 0개 완료
11560, 0개 완료
11590, 0개 완료
11620, 0개 완료
11650, 0개 완료
11680, 0개 완료
11710, 0개 완료
11740, 0개 완료
41110, 0개 완료
41111, 0개 

In [12]:
offi_trade

,deal_amt,build_y,deal_y,deal_m,deal_day,dong,apt_nm,area_exc,region_num,region_code,floor,sido,sigungu
0,"36,000",2008,2020,1,7,사직동,광화문 풍림스페이스본,39.54,9,11110,8,서울특별시,종로구
1,"33,000",2008,2020,1,17,사직동,광화문 풍림스페이스본,39.54,9,11110,2,서울특별시,종로구
2,"48,000",2004,2020,1,2,내수동,경희궁의 아침 2단지,51.02,71,11110,6,서울특별시,종로구
3,"28,500",2004,2020,1,14,내수동,경희궁의 아침 2단지,32.43,71,11110,13,서울특별시,종로구
4,"27,000",2004,2020,1,15,내수동,광화문시대,34.9,74,11110,12,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,"15,200",2018,2020,12,14,용문면 다문리,미르텔 B동,51.63,342-12,41830,4,경기도,양평군
9,"13,000",2018,2020,12,14,용문면 다문리,미르텔 B동,44,342-12,41830,4,경기도,양평군
10,"24,300",2018,2020,12,14,용문면 다문리,미르텔 B동,83.81,342-12,41830,5,경기도,양평군
11,"8,500",2018,2020,12,14,용문면 다문리,미르텔 B동,29.07,342-12,41830,2,경기도,양평군


In [13]:
offi_trade.to_csv('offi_trade_2019.csv', index=False)